In [12]:
import data.databaseManager as dbMgr
import pandas as pd

In [13]:
gamelog_data = dbMgr.query("""select playerid, timeid, stat, value from gameLog gl
                join games g on gl.gameid = g.gameid""")

In [14]:
modes = {'HALF': .5,
        'FULL': 1,
        'STD': 0}
scoring_settings = {'fumbles_lost': -2,
                   'pass_int': -1,
                    'pass_yds': .04,
                   'pass_td': 4,
                   'rec': None, # to be overriden
                   'rec_td': 6,
                   'rec_yds': .1,
                   'rush_td': 6,
                   'rush_yds': .1,}
pivoted_data = gamelog_data.pivot_table(index=['timeid', 'playerid'], columns='stat', values='value')
filtered_data = pivoted_data[list(scoring_settings.keys())]
recon_data = {}
for mode in modes:
    scoring_settings['rec'] = modes[mode]
    recon_data[mode] = (filtered_data * pd.Series(scoring_settings)).sum(axis=1)
recon_data = pd.DataFrame(recon_data)

In [15]:
contest_data = dbMgr.query("""select platform, playerid, timeid, points from contestPlayers cp
                            join contestStructure c on cp.contestid = c.contestid
                            where position not in ('K', 'DST')""")

In [16]:
platforms = contest_data['platform'].unique()

In [17]:
contestSettings = {'fanduel': 'HALF',
'draftkings': 'FULL',
'yahoo': 'HALF'}

In [18]:
recon_stats = {}
for plat in platforms:
    platform_data = contest_data[contest_data['platform'] == plat]
    recon_platform = platform_data.set_index(['timeid', 'playerid'])['points']
    iter_data = recon_data.copy()
    iter_data['recon'] = recon_platform.reindex(iter_data.index)
    iter_data = iter_data.dropna()
    if plat == 'fanduel':
        fanduel_data = iter_data
    for col in modes.keys():
        recon_stats[(plat, col)] = (iter_data[col] == iter_data['recon'])

In [19]:
all_stats = pd.DataFrame(recon_stats).groupby(level=0).sum()

In [20]:
for plat in platforms:
    assert((all_stats[plat][contestSettings[plat]] == all_stats[plat].max(axis=1)).all())

In [21]:
pid = fanduel_data.groupby('playerid')

In [23]:
recon_points = ((pid['recon'].sum() - pid['HALF'].sum()).abs() / pid['recon'].count())

In [33]:
assert(not ((pid['recon'].count() > 5) & (recon_points > 1.0)).any())